In [1]:
import sys
sys.path.insert(0, '../BO')

from gp import *
import numpy as np
from sklearn.datasets import fetch_mldata
from LG import *# Load and format the training and testing data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

loss = train_logisitc_regression_(mnist)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# set parameters
bounds = np.array([[-15., 0.0], [5, 200], [20, 2000], [0, 1]])
# maxeps specifies how many BOs we want to run
maxeps = 10
# n_iters is number of iterations we want for each run of BO
n_iters = 100
acqui_eva_num = 5
n_pre_samples = 3
greater_is_better = False
y_list = list()
coor_sigma = 1 * np.array([0.2,0.2,0.2,0.2,0.2,0.2])
burn_in = 10
input_dimension = 4
mode = 'MAP'
acqui_mode = 'MCMC'
acqui_sample_num = 5
process_sample_mode = 'abs'
prior_mode ='normal_prior'
likelihood_mode = 'normal_likelihood'

In [3]:
y_list = list()

for i in range(n_iters):
    y_list.append(list())


for j in range(maxeps):
    print ('Running %d episode' % (j + 1))
    xp, yp = bayesian_optimisation(coor_sigma = coor_sigma, 
                                   burn_in = burn_in, 
                                   input_dimension = input_dimension,
                                   n_iters = n_iters, 
                                   sample_loss = loss, 
                                   bounds = bounds, 
                                   n_pre_samples = n_pre_samples, 
                                   acqui_eva_num = acqui_eva_num,  
                                   greater_is_better=greater_is_better, 
                                   mode = mode, 
                                   acqui_mode = acqui_mode, 
                                   acqui_sample_num = acqui_sample_num,   
                                   process_sample_mode = process_sample_mode, 
                                   prior_mode = prior_mode, 
                                   likelihood_mode = likelihood_mode)
    y_train = yp[n_pre_samples:]
    for idx in range(len(y_train)):
        if idx == 0:
            y_list[idx].append(y_train[idx])
        else:
            y_list[idx].append(np.min(y_train[:idx]))

Running 1 episode
Start presampling...
	Learning rate: 0.000267203077653, training epochs: 164, batch size: 734, beta 0.464636692625
		 Accuracy: 0.8187
	Learning rate: 8.88920894801e-07, training epochs: 6, batch size: 1445, beta 0.244132031767
		 Accuracy: 0.6711
	Learning rate: 0.000264512612871, training epochs: 146, batch size: 1538, beta 0.566349605012
		 Accuracy: 0.7936
Presampling finished.
	Learning rate: 0.000266912778665, training epochs: 93, batch size: 1490, beta 0.542394162823
		 Accuracy: 0.7779
	Learning rate: 0.312132117181, training epochs: 160, batch size: 1517, beta 0.797582436689
		 Accuracy: 0.7789
	Learning rate: 7.12983642407e-06, training epochs: 40, batch size: 446, beta 0.957427653177
		 Accuracy: 0.6829
	Learning rate: 0.00015153758336, training epochs: 112, batch size: 442, beta 0.298522866735
		 Accuracy: 0.8132
	Learning rate: 2.96464193097e-05, training epochs: 105, batch size: 1042, beta 0.689178294081
		 Accuracy: 0.714
	Learning rate: 0.0001029205421

KeyboardInterrupt: 

In [ ]:
mean_list, std_list = return_mean_std(y_list)

In [ ]:
import matplotlib.pyplot as plt
x = [i+1 for i in range(len(y_list))]

plt.errorbar(x, mean_list, yerr = std_list, fmt = '-')

plt.show()

In [ ]:
import pickle
opt_dict = {}
opt_dict['y_list'] = y_list
with open('LR_MCMC_ABS.pkl','wb') as file:
    pickle.dump(opt_dict, file)

In [ ]:
with open('LR_MCMC_ABS.pkl', 'rb') as pickle_file:
    data1 = pickle.load(pickle_file)